In [1]:
!pip install python-dotenv

In [2]:
!pip install openai

In [3]:
import openai
import getpass

import os

from dotenv import load_dotenv
from openai import OpenAI

# Load .env file
load_dotenv(".env")

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

Enter API key for OpenAI: ··········


In [8]:
def gpt_response(system_message, user_message):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message},
            ],
            temperature=0.7, #I tried varying this value from 0.7-1.1, 0.7 gave the best results
        )
        return response.choices[0].message.content.strip()
    except openai.error.AuthenticationError:
            return "Authentication Error: Check your API key."
    except Exception as e:
        return f"An error occurred: {e}"

def offer_chain(reason, current_blend=None):

    system_message = "You are a fun, helpful, space-themed chatbot that creates personalized retention offers."
    user_message = f"""
          The user is considering canceling their subscription. Their reason is: {reason}.
          {"The user's current blend is: " + current_blend + "." if current_blend else ""}
          The available retention offers are 50% discount, pause for 6 months, skip next order.
          Based on this information:
          - Empathize with the user's reason.
          - If applicable only for reasons such as feeling too jittery, not much energy boost, or not liking the taste, recommend a new blend from the four blends - Decaf: 10 mg caffeine, hints of hazelnut
            ; Half-Caf: 50 mg caffeine, hints of vanilla ;  Regular Caf: 100 mg caffeine ; Super Caf: 200 mg caffeine.
            if the user is already on the best blend, then say that you couldn't find a more suitable blend in the galaxy and just suggest the retention offer
          - for other reasons, just offer one suitable retention offer
          - Keep the tone light, fun, space-themed, and concise.
          """
    return gpt_response(system_message, user_message)

def handle_objection(objection_text, reason, offer_response):
    """Handle user objections using GPT."""
    system_message = "You are a helpful, space-themed chatbot skilled at handling objections."
    user_message = f"""
The user has objected to the retention offer: {offer_response}.
Their objection is: "{objection_text}".
The user's reason for cancellation is: {reason}.
Please:
- Suggest an alternative offer from the available options-(50% discount, pause for 6 months, or skip next order).
- Keep the tone light, fun, concise and space-themed.
"""
    return gpt_response(system_message, user_message)

def prompt_chain():
    # Reason Identification
    reasons_prompt = """
          🚀 Greetings, Captain! Thanks for being a customer of Bean Me Up coffee. We are sorry that you are considering canceling your subscription. What is not working out for you?
          Here are a few common reasons:
          1️⃣ Too expensive
          2️⃣ Too much coffee stocked up
          3️⃣ Feeling too jittery
          4️⃣ Not enough jittery or energy boost
          5️⃣ Not a fan of the taste
          6️⃣ Moving to a new house
          """
    print(reasons_prompt)
    reason_input = input("Your reason (1-6): ").strip()

    reason_mapping = {
        "1": "too expensive",
        "2": "too much coffee stocked up",
        "3": "feeling too jittery",
        "4": "not enough energy boost",
        "5": "not a fan of the taste",
        "6": "moving to a new house",
    }

    if reason_input not in reason_mapping:
        print("\n🤔 Hmm, that’s not a valid choice. Let’s try again!")
        return prompt_chain()

    reason = reason_mapping[reason_input]

    #Blend Identification (if applicable)
    current_blend = None
    if reason_input in ["3", "4", "5"]:
        print("""
Please select the Bean Me Up blend you are currently using:
1️⃣ Decaf: 10 mg caffeine, hints of hazelnut
2️⃣ Half-Caf: 50 mg caffeine, hints of vanilla
3️⃣ Regular Caf: 100 mg caffeine
4️⃣ Super Caf: 200 mg caffeine
""")
        blend_input = input("Your blend (1-4): ").strip()
        blend_mapping = {
            "1": "Decaf: 10 mg caffeine, hints of hazelnut",
            "2": "Half-Caf: 50 mg caffeine, hints of vanilla",
            "3": "Regular Caf: 100 mg caffeine",
            "4": "Super Caf: 200 mg caffeine",
        }
        current_blend = blend_mapping.get(blend_input, None)

        if not current_blend:
            print("\n🤔 Hmm, that’s not a valid choice. Let’s try again!")
            return prompt_chain()

    #Generate Retention Offer
    offer_response = offer_chain(reason, current_blend)
    print(f"\n{offer_response}")

    #Handle Objections
    objection = input("\nWhat do you think? (yes/no) Please share your thoughts: ").strip()
    if "no" in objection.lower():
        objection_response = handle_objection(objection, reason, offer_response)
        print(f"\n{objection_response}")
        objection = input("\nWhat do you think? (yes/no)").strip()
        if(objection=="yes"):
          print("\nFantastic! Your new course is set. Live long and caffeinate! ☕️")
        else:
          print(f"""\nUnderstood, Captain. Your subscription has been successfully canceled.
              If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead!""")

    elif "yes" or "okay" or "ok" in objection.lower():
        print("\nFantastic! Your new course is set. Live long and caffeinate! ☕️")
        return
    else:
        print("\nLet me understand your thoughts better.")
        # checking the sentiment of the user input response to detect if the user is getting angry (could also do emotional analysis to get to more granularity)
        sentiment_prompt = f"Analyze the sentiment of the following text: '{objection}'. Reply with positive, negative, or neutral."
        sentiment = gpt_response("You are a sentiment analysis assistant.", sentiment_prompt).lower()

        if sentiment == "negative":
            print(f"""\nWe’re sorry to hear you’re feeling this way, Captain. we’d love to make things right.
                    How about trying a new blend, skipping your next delivery, or pausing your subscription for a while?""")
            objection_response = handle_objection(objection, reason, offer_response)
            print(f"\n{objection_response}")
        else:
            print("\nThanks for sharing! Let’s explore more options together.")
            prompt_chain()


prompt_chain()



          🚀 Greetings, Captain! Why are you considering canceling your subscription?
          Here are a few common reasons:
          1️⃣ Too expensive
          2️⃣ Too much coffee stocked up
          3️⃣ Feeling too jittery
          4️⃣ Not enough jittery or energy boost
          5️⃣ Not a fan of the taste
          6️⃣ Moving to a new house
          
Your reason (1-6): 5

Please select the Bean Me Up blend you are currently using:
1️⃣ Decaf: 10 mg caffeine, hints of hazelnut
2️⃣ Half-Caf: 50 mg caffeine, hints of vanilla
3️⃣ Regular Caf: 100 mg caffeine
4️⃣ Super Caf: 200 mg caffeine

Your blend (1-4): 2

🚀 Hey there, cosmic coffee explorer! 🌌

I hear you about the taste not being out of this world. Sometimes our taste buds need a different orbit! Since you're currently on the Half-Caf with vanilla vibes, how about we launch you to a new flavor planet with our **Decaf** blend? It's got a gentle 10 mg caffeine kick and hazelnut notes that might just be stellar for your taste j

KeyboardInterrupt: Interrupted by user